In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
import scipy as sc
import scipy.sparse as sp
from sksparse.cholmod import cholesky

# Reading and Plotting the Mesh

In [2]:
# Reading Triangle Mesh and Constraints
v, f = igl.read_triangle_mesh('data/woody-hi.off')
labels = np.load('data/woody-hi.label.npy').astype(int)

#v, f = igl.read_triangle_mesh('data/hand.off')
#labels = np.load('data/hand.label.npy').astype(int)

#v, f = igl.read_triangle_mesh('data/cactus.off')
#labels = np.load('data/cactus.label.npy').astype(int)

#v, f = igl.read_triangle_mesh('data/camel_head.off')
#labels = np.load('data/camel_head.label.npy').astype(int)

v -= v.min(axis=0)
v /= v.max()

# Ploting the original Mesh
mp.plot(v, f, c=labels, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

# Step 1: Removal of high-frequency details

In [3]:
# Indices of the faces that will be deformed
fixed_indices = np.where(labels > 0)[0]

# Indices of the faces that will not be deformed
free_indices = np.where(labels == 0)[0]

# Defining the Bi-Laplacian Operator Matrix
M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
L_w = igl.cotmatrix(v, f)
M_inv = sc.sparse.diags(1 / M.diagonal())
K = L_w.T @ M_inv @ L_w

K_ff = K[free_indices,:][:,free_indices]
K_ff = sp.csc_matrix(K_ff)
K_fc = K[free_indices,:][:,fixed_indices]

# Cholseky factorization
factor = cholesky(K_ff)

<h3>Smmothing Surface</h3>

In [4]:
# Function that smooth the surface
def smooth_surface(v):
    
    # Defining v_smooth
    v_smooth = v.copy()
    
    # Defining the b vector
    b = np.zeros([v.shape[0],3])
    
    b[free_indices,:] = b[free_indices,:] - K_fc @ v_smooth[fixed_indices,:]
    v_smooth[free_indices,:] = factor(b[free_indices,:])
    
    return v_smooth

<h3>Encoding Displacement Details</h3>

In [5]:
def encoding_displacement(v, v_smooth, f):
    # Computing the displacement vector
    d_i = v - v_smooth

    # Calculating normals from the smooth surface
    n_i = igl.per_vertex_normals(v_smooth, f)

    # Neighboring vertices
    neighbor_vertices = igl.adjacency_list(f)

    # Calculating projected edges
    projected_edges = np.zeros_like(n_i)
    indexes = []

    for j in range(len(neighbor_vertices)):
        distance = -10000
        indx = -1
        for neighbor_index in neighbor_vertices[j]:
            e_i = v_smooth[neighbor_index,:] - v_smooth[j,:] # calculates each edge
            par_proj = np.dot(n_i[j,:],e_i)* n_i[j,:] # paralell projection to normal
            per_proj =  e_i - par_proj # perpendicular projection of edge to normal
            if (np.linalg.norm(per_proj) > distance):
                distance = np.linalg.norm(per_proj) 
                projected_edges[j,:] =  per_proj
                indx = neighbor_index
        indexes.append(indx)

    # Defining the Basis for surface B
    x_i = projected_edges
    x_i /= np.linalg.norm(x_i, axis=1)[:,None]
    y_i = np.cross(n_i,x_i)
    
    # Constants
    d_i_c = np.zeros_like(v)

    for i in range(n_i.shape[0]):
        d_i_c[i,0] = d_i[i,:].dot(x_i[i,:])
        d_i_c[i,1] = d_i[i,:].dot(y_i[i,:])
        d_i_c[i,2] = d_i[i,:].dot(n_i[i,:])
        
    return [indexes, d_i_c]

<h3>Calculating Smooth Surface and Encoding Displacement Details</h3>

In [6]:
# Smoothing the surface by removing high frequency details
v_smooth = smooth_surface(v)

# Calculating Indexes for longest projected edge and the Displacement details
indexes, displacement_cosntants = encoding_displacement(v, v_smooth, f)

# Step 2. Deforming the Smooth Mesh

In [7]:
handle_vertex_positions = v_smooth.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))

# Functions that handle the deformation
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
    
def position_deformer(target_pos):
    v_deformed = smooth_surface(target_pos)
    v_transformed = transfering_displacements(v_deformed, indexes, displacement_cosntants)
    return v_transformed

pos_f.deformer = position_deformer

# Step 3: Transferring high-frequency details to the deformed surface

<h3>Function that transfer the displacement details back to deformed surface</h3>

In [8]:
def transfering_displacements(v_deformed, indexes, constants):
    
    # Getting the displacement constants
    d_i_x = constants[:,0]
    d_i_y = constants[:,1]
    d_i_z = constants[:,2]
    
    # Calculating normals for deformed surface
    n_i_1 = igl.per_vertex_normals(v_deformed, f)
    projected_vertex_1 = np.zeros_like(n_i_1)

    edges = v_deformed[indexes] - v_deformed

    for j in range(edges.shape[0]):
        e_i_1 = edges[j,:]
        par_proj_1 = np.dot(n_i_1[j,:],e_i_1)*n_i_1[j,:] #parallel proj
        per_proj_1 =  e_i_1 - par_proj_1 
        projected_vertex_1[j,:] = per_proj_1

    x_i_1 = projected_vertex_1
    x_i_1 /= np.linalg.norm(x_i_1, axis=1)[:,None]
    y_i_1 = np.cross(n_i_1,x_i_1)

    d_i_1 = np.zeros_like(n_i_1)
    for j in range(d_i_1.shape[0]):
        d_i_1[j] = d_i_x[j] * x_i_1[j] + d_i_y[j] * y_i_1[j] + d_i_z[j] * n_i_1[j] 

    v_transformed = v_deformed + d_i_1
    return v_transformed

# Visualization of Deformed Mesh

In [9]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [10]:
## Widget UI
p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4, 5), value=1), FloatSlider(value=0.0, desc…

<function __main__.pos_f(s, x, y, z, α, β, γ)>